# KCMPC

Cite

* [Hongkai Dai: testJump](https://github.com/RobotLocomotion/drake/blob/last_sha_with_original_matlab/drake/matlab/solvers/trajectoryOptimization/dev/testJump.m)
* [Hongkai Dai: How to improve the trajectory optimization results](https://stackoverflow.com/questions/68698187/how-to-improve-the-trajectory-optimization-results)
* [casadi: Opti stack](https://web.casadi.org/docs/#document-opti)

In [1]:
import time
import unittest

import casadi
import example_robot_data as robex
import numpy as np
import pinocchio as pin
from numpy.linalg import norm
from pinocchio import casadi as cpin
from scipy.optimize import fmin_bfgs

from utils.meshcat_viewer_wrapper import MeshcatVisualizer

### import model

In [2]:
robot = robex.load("talos_legs")
# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
viz.viewer.jupyter_cell()

/home/matheecs/Code/jnrh2023/myenv/lib/python3.10/site-packages/cmeel.prefix/lib/python3.10/site-packages/example_robot_data/robots_loader.py:319: UserWarning: Deprecated member. Use Frame.parentJoint instead.
  if f.parent < legMaxId:


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [3]:
from pinocchio.utils import eye

model = robot.model
left_jID = model.getJointId("leg_left_6_joint")
right_jID = model.getJointId("leg_right_6_joint")
model.addFrame(pin.Frame('left_contact_point_1', left_jID, pin.SE3(
    eye(3), np.matrix([.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_2', left_jID, pin.SE3(
    eye(3), np.matrix([-.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_3', left_jID, pin.SE3(
    eye(3), np.matrix([-.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('left_contact_point_4', left_jID, pin.SE3(
    eye(3), np.matrix([.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_1', right_jID, pin.SE3(
    eye(3), np.matrix([.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_2', right_jID, pin.SE3(
    eye(3), np.matrix([-.1,  .06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_3', right_jID, pin.SE3(
    eye(3), np.matrix([-.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
model.addFrame(pin.Frame('right_contact_point_4', right_jID, pin.SE3(
    eye(3), np.matrix([.1, -.06, -0.1]).T), pin.FrameType.OP_FRAME))
data = model.createData()

In [4]:
contact_names = ["left_contact_point_1", "left_contact_point_2", "left_contact_point_3", "left_contact_point_4",
                 "right_contact_point_1", "right_contact_point_2", "right_contact_point_3", "right_contact_point_4"]
# Add a vizualization for the target
for i in range(8):
    cID = "world/" + contact_names[i]
    viz.addSphere(cID, 0.02, [1.0, 0.2, 0.2, 0.5])
    pin.framesForwardKinematics(model, data, robot.q0)
    viz.applyConfiguration(cID, data.oMf[model.getFrameId(contact_names[i])])

### compute casadi graphs

In [5]:
# --- Casadi helpers
cmodel = cpin.Model(model)
cdata = cmodel.createData()

nq = model.nq
nv = model.nv
cq = casadi.SX.sym("q", nq, 1)
cv = casadi.SX.sym("v", nv, 1)
ct = casadi.SX.sym("t")

# Compute kinematics casadi graphs
cpin.forwardKinematics(cmodel, cdata, cq)
cpin.updateFramePlacements(cmodel, cdata)
# Compute com casadi graphs
cpin.centerOfMass(cmodel, cdata, cq)
# Compute centroidal momentum casadi graphs
cpin.ccrba(cmodel, cdata, cq, cv)
print("Get Graphs Done.")

Get Graphs Done.


In [6]:
# Sym graph for the integration operation q,dq -> q(+)dq = model.integrate(q,dq)
cintegrate = casadi.Function(
    "integrate",
    [cq, cv],
    [cpin.integrate(cmodel, cq, cv)]
)

cnext = casadi.Function(
    "next",
    [cq, cv, ct],
    [cpin.integrate(cmodel, cq, cv * ct)]
)

In [7]:
contacts_fk_list = []
for i in range(8):
    endEffector_ID = model.getFrameId(contact_names[i])
    contacts_fk_list.append(casadi.Function(
        contact_names[i]+"Fun",
        [cq],
        [cdata.oMf[endEffector_ID].translation]
    ))

In [8]:
h = casadi.Function(
    "hFun",
    [cq, cv],
    [cdata.hg.angular]
)

In [9]:
com = casadi.Function(
    "comFun",
    [cq],
    [cdata.com[0]]
)

In [10]:
print("com = ", com(robot.q0))
print("h = ", h(robot.q0, np.zeros(nv)))
for i in range(8):
    print(contact_names[i], contacts_fk_list[i](robot.q0))

com =  [0.0034147, 0.00228237, 0.618288]
h =  [0, 0, 0]
left_contact_point_1 [0.091153, 0.144829, 0.00689549]
left_contact_point_2 [-0.108847, 0.144829, 0.00689549]
left_contact_point_3 [-0.108847, 0.0248293, 0.00710045]
left_contact_point_4 [0.091153, 0.0248293, 0.00710045]
right_contact_point_1 [0.091153, -0.0251709, 0.00689549]
right_contact_point_2 [-0.108847, -0.0251709, 0.00689549]
right_contact_point_3 [-0.108847, -0.145171, 0.00710045]
right_contact_point_4 [0.091153, -0.145171, 0.00710045]


### add variables

In [11]:
nT = 17
opti = casadi.Opti()


var_dq = [opti.variable(robot.nv) for t in range(nT)]
var_q = [cintegrate(robot.q0, dq) for dq in var_dq]
var_v = [opti.variable(robot.nv) for t in range(nT)]
var_t = [opti.variable(1) for t in range(nT)]


var_r = [com(q) for q in var_q]
var_dr = [opti.variable(3) for t in range(nT)]
var_ddr = [opti.variable(3) for t in range(nT)]


var_c = [[contacts_fk_list[i](q) for i in range(8)]
         for q in var_q]   # len(var_c) == nT
var_F = [[opti.variable(3) for i in range(8)]
         for t in range(nT)]  # len(var_F) == nT


var_h = [h(var_q[t], var_v[t]) for t in range(nT)]
var_dh = [opti.variable(3) for t in range(nT)]

In [12]:
m = pin.computeTotalMass(robot.model)

In [13]:
g = robot.model.gravity981

### add constraints

In [14]:
# 7a
for k in range(0, nT):
    opti.subject_to(m * var_ddr[k] == sum(var_F[k]) * m * norm(g) + m * g)

In [15]:
# 7b
for k in range(1, nT):
    opti.subject_to(var_dh[k] == m * (casadi.cross(var_c[k][0] - var_r[k], var_F[k][0] * m * norm(g))
                                      + casadi.cross(var_c[k][0] - var_r[k], var_F[k][1] * m * norm(g))
                                      + casadi.cross(var_c[k][0] - var_r[k], var_F[k][2] * m * norm(g))
                                      + casadi.cross(var_c[k][0] - var_r[k], var_F[k][3] * m * norm(g))
                                      + casadi.cross(var_c[k][0] - var_r[k], var_F[k][4] * m * norm(g))
                                      + casadi.cross(var_c[k][0] - var_r[k], var_F[k][5] * m * norm(g))
                                      + casadi.cross(var_c[k][0] - var_r[k], var_F[k][6] * m * norm(g))
                                      + casadi.cross(var_c[k][0] - var_r[k], var_F[k][7] * m * norm(g))))

In [16]:
# 7c
for k in range(0, nT):
    opti.subject_to(var_h[k] == h(var_q[k], var_v[k]))

In [17]:
# 7d
for k in range(0, nT):
    opti.subject_to(var_q[k] == cnext(var_q[k-1], var_v[k], var_t[k]))

In [18]:
# 7e
for k in range(1, nT):
    opti.subject_to(var_h[k] - var_h[k-1] == var_dh[k] * var_t[k])

In [19]:
# 7f
for k in range(1, nT):
    opti.subject_to(var_r[k] - var_r[k-1] == 0.5 *
                    (var_dr[k] + var_dr[k-1]) * var_t[k])
# 7g
for k in range(1, nT):
    opti.subject_to(var_dr[k] - var_dr[k-1] == var_ddr[k] * var_t[k])

#### contact sequence

In [20]:
def get_stance_schedule():
    in_stance = np.zeros((len(contact_names), nT))
    t = 0
    for ci in range(8):
        in_stance[ci, t:] = 1
    t = 4
    for ci in range(8):
        if ci == 1 or ci == 2 or ci == 5 or ci == 6:
            in_stance[ci, t:] = 0
    t = 6
    for ci in range(8):
        in_stance[ci, t:] = 0
    t = 11
    for ci in range(8):
        if ci == 1 or ci == 2 or ci == 5 or ci == 6:
            in_stance[ci, t:] = 1
    t = 13
    for ci in range(8):
        in_stance[ci, t:] = 1

    return in_stance


in_stance = get_stance_schedule()
print(in_stance)

[[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.]]


In [21]:
# 7j
for k in range(1, nT):
    for ci in range(8):
        if in_stance[ci, k]:
            # stance case
            opti.subject_to(var_c[k][ci] == contacts_fk_list[ci](robot.q0))
        else:
            # swing case: contact height is above ground
            opti.subject_to(var_c[k][ci][2] >= 0.0)

In [22]:
# 7k
t_bound = [0.5, 2.0]
for k in range(1, nT):
    opti.subject_to(opti.bounded(robot.model.lowerPositionLimit,
                                 var_q[k], robot.model.upperPositionLimit))
    opti.subject_to(opti.bounded(-robot.model.velocityLimit,
                                 var_v[k], robot.model.velocityLimit))

    # friction cone
    mu = 1
    for ci in range(8):
        opti.subject_to(opti.bounded(0, var_F[k][ci][2], 4 * in_stance[ci][k]))
        opti.subject_to(-var_F[k][ci][0] <= mu*var_F[k][ci][2])
        opti.subject_to(var_F[k][ci][0] <= mu*var_F[k][ci][2])
        opti.subject_to(-var_F[k][ci][1] <= mu*var_F[k][ci][2])
        opti.subject_to(var_F[k][ci][1] <= mu*var_F[k][ci][2])

    # time duration
    opti.subject_to(opti.bounded(
        0.5 * t_bound[0]/nT, var_t[k], 2.0 * t_bound[1]/nT))
opti.subject_to(opti.bounded(t_bound[0], sum(var_t), t_bound[1]))

#### initial and terminal constraint

In [23]:
opti.subject_to(var_q[0] == robot.q0)
opti.subject_to(var_v[0] == np.zeros(robot.nv))
opti.subject_to(var_q[-1] == robot.q0)
opti.subject_to(var_v[-1] == np.zeros(robot.nv))

opti.subject_to(var_t[0] == 0)
opti.subject_to(var_dr[0] == np.zeros(3))
opti.subject_to(var_ddr[0] == np.zeros(3))
opti.subject_to(var_dh[0] == np.zeros(3))

### add costs

In [24]:
totalcost = 0
for k in range(1, nT):
    totalcost += casadi.sumsqr(var_dq[k]) * 1.0 * var_t[k]
    totalcost += casadi.sumsqr(var_v[k]) * 0.1 * var_t[k]
    totalcost += casadi.sumsqr(var_ddr[k]) * 1.0 * var_t[k]
    for k in range(8):
        totalcost += casadi.sumsqr(var_F[k][ci]) * 1.0e-4 * var_t[k]

### set initial value and solve it

In [25]:
for k in range(1, nT):
    opti.set_initial(var_t[k], 0.06)
    opti.set_initial(var_dq[k], np.zeros(robot.nv))
    opti.set_initial(var_v[k], np.zeros(robot.nv))

for k in range(nT):
    for ci in range(8):
        opti.set_initial(var_F[k][ci], np.zeros(3))
        if in_stance[ci, k]:
            opti.set_initial(var_F[k][ci][2], 1.0 / sum(in_stance[:, k]))

In [26]:
opti.minimize(totalcost)
opti.solver("ipopt", {"expand": True}, {"max_iter": 7000,
            "fixed_variable_treatment": "make_constraint"})
try:
    sol = opti.solve()
except:
    print("ERROR in convergence, plotting debug info.")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:    14834
Number of nonzeros in inequality constraint Jacobian.:     2761
Number of nonzeros in Lagrangian Hessian.............:    17024

Total number of variables............................:     1190
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      973
Total number of inequality c

In [27]:
sol_q = [pin.integrate(model, robot.q0, opti.value(dq)) for dq in var_dq]

In [28]:
def displayScene(q, dt=1e-1):
    pin.framesForwardKinematics(model, data, q)
    viz.display(q)
    time.sleep(dt)


def displayTraj(qs, dt=1e-2):
    for q in qs[1:]:
        displayScene(q, dt=dt)

In [29]:
viz.viewer.jupyter_cell()

In [40]:
print("***** Display the resulting trajectory ...")
displayScene(robot.q0, 1)
displayTraj(sol_q, 0.2)

***** Display the resulting trajectory ...


In [38]:
for t in var_t:
    print("{:.2f}".format(opti.value(t)))

0.00
0.06
0.01
0.01
0.02
0.01
0.13
0.01
0.01
0.01
0.10
0.01
0.01
0.01
0.01
0.03
0.01


In [32]:
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [33]:
[opti.value(var_F[i][0]) for i in range(nT)]

[array([0.   , 0.   , 0.125]),
 array([-0.001, -0.002,  0.002]),
 array([0.001, 0.003, 0.003]),
 array([-0.052, -0.088,  1.229]),
 array([0.001, 0.005, 0.005]),
 array([-0.093, -0.151,  1.984]),
 array([0., 0., 0.]),
 array([0., 0., 0.]),
 array([0., 0., 0.]),
 array([ 0., -0.,  0.]),
 array([ 0., -0.,  0.]),
 array([-0., -0.,  0.]),
 array([-0., -0.,  0.]),
 array([ 0.001, -0.   ,  0.001]),
 array([-0.077, -0.125,  0.616]),
 array([-0., -0.,  0.]),
 array([-0.002, -0.002,  0.002])]

In [34]:
[opti.value(var_ddr[i]) for i in range(nT)]

[array([ 0.,  0., -0.]),
 array([-0.099, -0.118, -9.692]),
 array([ 0.103,  0.201, -9.609]),
 array([-4.097, -6.931, 86.653]),
 array([ 0.057,  0.188, -9.622]),
 array([-3.656, -5.927, 68.039]),
 array([ 0.  ,  0.  , -9.81]),
 array([ 0.  ,  0.  , -9.81]),
 array([ 0.  ,  0.  , -9.81]),
 array([ 0.  , -0.  , -9.81]),
 array([ 0.  , -0.  , -9.81]),
 array([ -7.129, -11.642,  60.004]),
 array([-3.789, -6.234, 25.199]),
 array([ 0.078, -0.02 , -9.732]),
 array([-6.043, -9.83 , 38.559]),
 array([-0.029, -0.013, -9.781]),
 array([-0.194, -0.125, -9.616])]

In [35]:
if False:
    opti.debug.show_infeasibilities()